In [ ]:
import sys
import os
# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory of 'scrapping' to the system path
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'scrapping')))
from text_cleaner import read_and_clean_adrs

from nltk.corpus import stopwords
from markdown2 import markdown
import openai
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance, OpenAI
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
import datamapplot
import pandas as pd
import seaborn

import warnings
warnings.filterwarnings('ignore')

# load environment variables
from dotenv import load_dotenv
load_dotenv()

# Path to the ADR directory
adr_directory = "../../data/ADRs-Updated"

In [ ]:
from datetime import datetime

def parse_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')

# Iterate over all files in the directory
metadata_path = "../../data/ADR-Study-Dataset-Metadata/repositories"
total_adr_files = 0
earliest_date = None
for filename in os.listdir(metadata_path):
    if filename.endswith('.json'):
        file_path = os.path.join(metadata_path, filename)
        # Open and read the JSON file
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            # Add the value of numAdrFiles to the total
            total_adr_files += data.get('numAdrFiles', 0)
            for adr_file in data.get('adrFiles', []):
                first_commit_date = parse_date(adr_file['firstCommit'])
                if earliest_date is None or first_commit_date < earliest_date:
                    earliest_date = first_commit_date
            
# Print the total number of ADR files
print(f'Total number of ADR files in metadata: {total_adr_files}')
print(f'Earliest commit date: {earliest_date}')

In [ ]:
# count total number of files in the ADR directory
total_adr_files = 0
for filename in os.listdir(adr_directory):
    if filename.endswith('.md'):
        total_adr_files += 1
print(f'Total number of ADR files in directory: {total_adr_files}')

# Most common terms in adr names

In [ ]:
file_name_word_count = {}
for file_name in os.listdir(adr_directory):
    words = file_name.split('-')
    words = [word.replace('.md', '') for word in words]
    # convert to lowercase
    words = [word.lower() for word in words]
    # remove the numbers
    words = [word for word in words if not word.isdigit()]
    # remove the word ADR
    words = [word for word in words if word.lower() != 'adr']
    # keep words longer than 2 characters
    words = [word for word in words if len(word) > 2]
    # use limatizer
    # remove stop words
    words = [word for word in words if word not in stop_words]
    for word in words:
        if word in file_name_word_count:
            file_name_word_count[word] += 1
        else:
            file_name_word_count[word] = 1

# Top 10 words in ADR file names
print("Top 10 words in ADR file names:")
for word, count in Counter(file_name_word_count).most_common(50):
    print(f"{word}: {count}")


## Average clean tokens (words) per ADR: 212.4236214605067

In [ ]:
# produce a distibution

## Most frequent words inside adrs

In [ ]:
all_words = []
for file_name in os.listdir(adr_directory):
    file_path = os.path.join(adr_directory, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        preprocessed_text, words = clean_text(text)
        all_words.extend(words)

word_freq = Counter(all_words)
print(word_freq.most_common(50))